In [2]:
import os
from triangulator import create_triangulator_from_files

import sys
sys.path.append('../')
from utils.file_utils import create_capture_from_json

calibration_folder = '../calibration/calibration_data/'
intr_L = os.path.join(calibration_folder, 'calibration_L_frames=623_error=1.5265531081871597.pkl')
intr_R = os.path.join(calibration_folder, 'calibration_R_frames=624_error=0.99089142253755.pkl')
orientation_L = '../calibration/orientation_4.pkl'
orientation_R = '../calibration/orientation_2.pkl'

triangulator = create_triangulator_from_files(
    intr_files=[intr_L, intr_R],
    orientation_files=[orientation_L, orientation_R]
)

Image size is (1920, 1080)
Image size is (1920, 1080)


In [3]:
import numpy as np

zero_point_L = (582, 739)
zero_point_R = (597, 855)

projection = triangulator.projection_matrices[0]
extr = triangulator.extrinsics[0]
cam_matr = triangulator.camera_matrices[0]

# get position with triangulators constructed projection matrix
position_proj = (projection @ [0, 0, 0, 1])
position_proj = position_proj / position_proj[-1]

# get position with triangulator extrinsics matrix
position_extr = (extr @ [0, 0, 0, 1])
position_extr = cam_matr @ position_extr[:3]
position_extr = position_extr / position_extr[-1]

# create inverse projection matrix
inv_proj = np.linalg.inv(extr)
position_inv = cam_matr @ inv_proj[0:3]
position_inv = position_inv @ [0, 0, 0, 1]
position_inv = position_inv / position_inv[-1]

print('position from projection matrix: ', position_proj)
print('position from extrinsics matrix: ', position_extr)
print('position from inverse matrix   : ', position_inv)

position from projection matrix:  [572.87374321 743.87228508   1.        ]
position from extrinsics matrix:  [-5.10290077e+02 -2.02182912e+03  1.00000000e+00]
position from inverse matrix   :  [572.87374321 743.87228508   1.        ]


In [ ]:
p1, p2, p3 = tuple(triangulator.projection_matrices[0])
print(p1)
print(p2)
print(p3)
print(triangulator.projection_matrices[0])

In [ ]:
caps = [
    create_capture_from_json(4, '../config/capture_params.json'),
    create_capture_from_json(2, '../config/capture_params.json')
]

In [1]:
import cv2


while True:
    idx = 0
    for cap in caps:
        ret, frame = cap.read()

        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        corners, ids, _ = cv2.aruco.detectMarkers(
            image=gray,
            dictionary=cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_50)
        )
        if ids is not None and 0 in ids:
            for corner_idx in range(len(ids)):
                if ids[corner_idx] == 0:
                    cv2.aruco.drawDetectedMarkers(
                        frame, 
                        corners=corners, 
                        ids=ids
                        )
                    break
        idx += 1
        cv2.imshow(f'cam{idx}', cv2.resize(src=frame, dsize=None, fx=0.35, fy=0.35))
    if cv2.waitKey(10) == 27:
        break


cv2.imshow('huh', frame)
cv2.waitKey(1000)
cv2.destroyAllWindows()

NameError: name 'caps' is not defined

In [48]:
for cap in caps:
    cap.release()

cv2.destroyAllWindows()